# Solving Sudoku with AI

This is my solution for Udacity's AI Nanodegree's Sudoku Solver project.

Also, for future refere

In [198]:
# Dependencies
from functools import reduce

In [278]:
# Simple Test Case for Validating Code

# Note: Both these test cases fail when the diagonal unit check is added.
tests = [
    {
        'input': '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..',
        'output': '483921657967345821251876493548132976729564138136798245372689514814253769695417382'
    },
    {
        'input': '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......',
        'output': ''
    }
]

In [200]:
# Base Row and Column Information
rows = 'ABCDEFGHI'
cols = '123456789'

In [201]:
def cross(a, b):
    """
    Calculate the cross product of two lists
    """
    return [ s + t for s in a for t in b ]

In [276]:
# Set of all possible Boxes in a Sudoku Game
boxes = cross(rows, cols)

# Calculate all Unit elements for Rows, Columns, and Squares
row_units = [ cross(r, cols) for r in rows ]
column_units = [ cross(rows, c) for c in cols ]
square_units = [ cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789') ]

# Calculate Diagonal Units
diagonal_units = [
    [ a + b for a, b in zip(rows, cols) ],
    [ a + b for a, b in zip(rows, reversed(cols)) ]
]

# Combined Unit List
unitlist = row_units + column_units + square_units # + diagonal_units

# Create Maps for Units by Unit Name and Units' Peers by Unit Name
units = dict((s, [u for u in unitlist if s in u ]) for s in boxes)
peers = dict((s, set(sum(units[s], [])) - set([s])) for s in boxes)

In [203]:
def display(values):
    """
    Print a 2-Dimensional representation of the given
    Sudoku board.
    """
    
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join([ '-' * (width * 3)] * 3)
    for r in rows:
        print(''.join(values[r + c].center(width) + ('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    return

In [204]:
def grid_values(grid):
    """
    Convert grid string into {<box>: <value>} dict with '123456789' value for empties.
    
    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    
    empty_box_val = '123456789'
    return { k: (empty_box_val if v == '.' else v) for k, v in zip(boxes, grid) }

In [205]:
def eliminate(values):
    """
    Eliminate values from peers of each box with a single value.
    
    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    
    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after elminating values.
    """
    
    solved_values = [ box for box in values.keys() if len(values[box]) == 1 ]
    for box in solved_values:
        val = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(val, '')
    return values

In [206]:
def only_choice(values):
    """
    Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after filling in only choices.
    """
    
    for box in values.keys():
        peer_values = reduce(lambda s, i: s.union(set(i)), map(lambda peer: values[peer], peers[box]), set())
        
        intersect_values = set(values[box]) - peer_values
        if len(intersect_values) == 1:
            values[box] = intersect_values.pop()

    return values

In [207]:
def naked_twins(values):
    """
    Use Naked Twins elimination strategy to remove invalid
    configurations from the grid.
    """
    vs = values.copy()
    
    for box in values.keys():
        for peer in peers[box]:
            if len(values[box]) == 2 and len(values[peer]) == 2 and set(values[box]) == set(values[peer]):
                for p in set(peers[box]).intersection(set(peers[peer])):
                    for digit in values[box]:
                        vs[p] = vs[p].replace(digit, '')
    
    return vs

In [208]:
def reduce_puzzle(values):
    """
    Reduce the given Sudoku puzzle by recursively eliminating and simplifying obvious choices.
    """
    
    stalled = False
    while not stalled:
        solved_values_before = len([ box for box in values.keys() if len(values[box]) == 1])
        
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        
        solved_values_after = len([ box for box in values.keys() if len(values[box]) == 1 ])
        stalled = solved_values_before == solved_values_after
        
        if len([ box for box in values.keys() if len(values[box]) == 0 ]):
            return False
        
    return values

In [209]:
def search(values):
    """
    Using depth-first search and propagation, create
    a search tree and solve the sudoku.
    """
    
    values = reduce_puzzle(values)
    if values == False:
        return False
    
    # Get all unfilled boxes
    unfilled = sorted([ box for box in values.keys() if len(values[box]) > 1 ])
    if len(unfilled) == 0:
        return values
    
    target_box = unfilled[0]
    
    for val in values[target_box]:
        vs = values.copy()
        vs[target_box] = val
        result = search(vs)
        if result != False:
            return result
    
    return False

In [220]:
def convert_output(values):
    return reduce(lambda s, i: s + i, [ values[box] for box in boxes ], '')

In [273]:
def validate_board(values):
    for unit in unitlist:
        unit_values = reduce(lambda s, y: s + y, sorted(map(lambda box: values[box], unit)), '')
        #print('Values: {}'.format(unit_values))
        assert unit_values == '123456789', "Unit={}, Expected '123456789', but got: '{}'".format(list(unit), unit_values)
    return

In [274]:
def test_case(test_input, expected_output):
    print("Running Test for Input: '{}'".format(test_input))
    values = grid_values(test_input)
    output = search(values)
    
    validate_board(output)
    
    print()
    print("Received Output: '{}'".format(convert_output(output)))
    print("Expected Output: '{}'".format(expected_output))
    
    print()
    display(output)
    # assert output == expected_output

In [277]:
test_case(tests[0]['input'], tests[0]['output'])

Running Test for Input: '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

Received Output: '483921657967345821251876493548132976729564138136798245372689514814253769695417382'
Expected Output: '483921657967345821251876493548132976729564138136798245372689514814253769695417382'

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


In [217]:
test_case(tests[1]['input'], tests[1]['output'])

Running Test for Input: '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'

Received Output: '417369825632158947958724316825437169791586432346912758289643571573291684164875293'
Expected Output: ''

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 
